In [5]:
import os, glob, math, warnings
import numpy as np, pandas as pd, xarray as xr
import tensorflow as tf
from sklearn.metrics import mean_squared_error
from tqdm.auto import tqdm
from datetime import datetime


2025-05-06 21:17:11.230821: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-06 21:17:11.263293: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746546431.298179 1390992 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746546431.308573 1390992 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1746546431.337499 1390992 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [24]:

# --- Constants ---
DATA_ROOT = "dataset"
MONTH_FOLDERS = ["Jun_Output", "Jul_Output", "Aug_Output"]
GHI_CSV = "dataset/Sample Dataset - ML Assignment - Sheet1.csv"

WINDOW = 1
SHIFT = 1
HORIZON = 1
MAX_GHI = 1100.0
SAT_SCALE = 1023.0
CHANNEL_VARS = ["IMG_MIR", "IMG_SWIR", "IMG_TIR1", "Sun_Elevation", "Sat_Elevation"]


In [ ]:

# --- Setup ---
np.random.seed(42)
tf.random.set_seed(42)
warnings.filterwarnings("ignore")

# --- Load GHI CSV ---
ghi_df = (pd.read_csv(GHI_CSV)
            .rename(columns={"Date ": "timestamp", "Observed GHI ": "GHI"}))
ghi_df["timestamp"] = pd.to_datetime(ghi_df["timestamp"])
ghi_dict = ghi_df.set_index("timestamp")["GHI"].to_dict()

# --- Load .nc sample for training ---
def load_day_nc(nc_path, vars_to_use):
    ds = xr.open_dataset(nc_path)
    daily_stack, ghi_labels = [], []
    for t_idx in range(len(ds.time)):
        t_stamp = pd.to_datetime(str(ds.time[t_idx].values))
        if t_stamp not in ghi_dict:
            continue
        channels = [ds[var].isel(time=t_idx).values.astype("float32") for var in vars_to_use]
        patch = np.stack(channels, axis=-1)
        daily_stack.append(patch)
        ghi_labels.append(ghi_dict[t_stamp])
    if len(daily_stack) >= (WINDOW + SHIFT):
        return np.stack(daily_stack), np.array(ghi_labels, dtype="float32")
    return None, None

# --- Build dataset ---
Xs, ys = [], []
all_nc_paths = []
for month in MONTH_FOLDERS:
    all_nc_paths += glob.glob(os.path.join(DATA_ROOT, month, "**", "*.nc"), recursive=True)

for nc_path in all_nc_paths:
    X_day, y_day = load_day_nc(nc_path, CHANNEL_VARS)
    if X_day is None: continue

    ghi_plane = y_day.reshape(-1, 1, 1, 1)
    ghi_plane = np.repeat(ghi_plane, 5, axis=1)
    ghi_plane = np.repeat(ghi_plane, 5, axis=2)
    X_day = np.concatenate([X_day, ghi_plane], axis=-1)

    for t0 in range(len(X_day) - WINDOW + 1):
        if t0 + WINDOW - SHIFT + HORIZON > len(y_day):
            continue
        Xs.append(X_day[t0:t0+WINDOW])
        ys.append(y_day[t0 + WINDOW - SHIFT : t0 + WINDOW - SHIFT + HORIZON])

X = np.stack(Xs).astype("float32") / SAT_SCALE
y = np.stack(ys).astype("float32") / MAX_GHI

# --- Train/val split ---
split = int(0.8 * len(X))
X_train, X_val = X[:split], X[split:]
y_train, y_val = y[:split], y[split:]

# --- Define ConvLSTM model ---
model = tf.keras.Sequential([
    tf.keras.layers.ConvLSTM2D(32, (3,3), activation='relu', return_sequences=True,
                               input_shape=(WINDOW, 5, 5, X.shape[-1]), padding="same"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ConvLSTM2D(16, (1,1), activation='relu', padding="same"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(HORIZON)
])
model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
              loss=tf.keras.losses.Huber(), metrics=["mae"])

# --- Callbacks ---
callbacks = [
    tf.keras.callbacks.ReduceLROnPlateau(patience=5, factor=0.5, verbose=1),
    tf.keras.callbacks.EarlyStopping(patience=12, restore_best_weights=True, verbose=1),
    tf.keras.callbacks.ModelCheckpoint("final_convLSTM_shift3_best.h5", save_best_only=True, verbose=1)
]

# --- Train model ---
model.fit(X_train, y_train,
          validation_data=(X_val, y_val),
          epochs=100, batch_size=16,
          callbacks=callbacks, verbose=1)



Epoch 1/100
119/119 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 0.1733 - mae: 0.4406
Epoch 1: val_loss improved from inf to 0.06234, saving model to final_convLSTM_shift3_best.h5


119/119 ━━━━━━━━━━━━━━━━━━━━ 11s 40ms/step - loss: 0.1725 - mae: 0.4396 - val_loss: 0.0623 - val_mae: 0.2767 - learning_rate: 0.0010
Epoch 2/100
119/119 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 0.0453 - mae: 0.2263
Epoch 2: val_loss improved from 0.06234 to 0.04431, saving model to final_convLSTM_shift3_best.h5


119/119 ━━━━━━━━━━━━━━━━━━━━ 4s 32ms/step - loss: 0.0453 - mae: 0.2262 - val_loss: 0.0443 - val_mae: 0.2256 - learning_rate: 0.0010
Epoch 3/100
117/119 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 0.0330 - mae: 0.1936
Epoch 3: val_loss improved from 0.04431 to 0.03498, saving model to final_convLSTM_shift3_best.h5


119/119 ━━━━━━━━━━━━━━━━━━━━ 4s 32ms/step - loss: 0.0330 - mae: 0.1936 - val_loss: 0.0350 - val_mae: 0.2018 - learning_rate: 0.0010
Epoch 4/100
119/119 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 0.0276 - mae: 0.1817
Epoch 4: val_loss improved from 0.03498 to 0.03073, saving model to final_convLSTM_shift3_best.h5


119/119 ━━━━━━━━━━━━━━━━━━━━ 4s 31ms/step - loss: 0.0276 - mae: 0.1817 - val_loss: 0.0307 - val_mae: 0.1931 - learning_rate: 0.0010
Epoch 5/100
119/119 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.0257 - mae: 0.1786
Epoch 5: val_loss improved from 0.03073 to 0.02896, saving model to final_convLSTM_shift3_best.h5


...

119/119 ━━━━━━━━━━━━━━━━━━━━ 4s 33ms/step - loss: 0.0252 - mae: 0.1797 - val_loss: 0.0278 - val_mae: 0.1907 - learning_rate: 0.0010
Epoch 10/100
118/119 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 0.0252 - mae: 0.1797
Epoch 10: val_loss improved from 0.02780 to 0.02778, saving model to final_convLSTM_shift3_best.h5


119/119 ━━━━━━━━━━━━━━━━━━━━ 4s 31ms/step - loss: 0.0252 - mae: 0.1797 - val_loss: 0.0278 - val_mae: 0.1907 - learning_rate: 0.0010
Epoch 11/100
119/119 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 0.0252 - mae: 0.1798
Epoch 11: val_loss improved from 0.02778 to 0.02777, saving model to final_convLSTM_shift3_best.h5


119/119 ━━━━━━━━━━━━━━━━━━━━ 4s 33ms/step - loss: 0.0252 - mae: 0.1798 - val_loss: 0.0278 - val_mae: 0.1907 - learning_rate: 0.0010
Epoch 12/100
119/119 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 0.0252 - mae: 0.1798
Epoch 12: val_loss improved from 0.02777 to 0.02776, saving model to final_convLSTM_shift3_best.h5


119/119 ━━━━━━━━━━━━━━━━━━━━ 4s 35ms/step - loss: 0.0252 - mae: 0.1798 - val_loss: 0.0278 - val_mae: 0.1907 - learning_rate: 0.0010
Epoch 13/100
119/119 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 0.0252 - mae: 0.1798
Epoch 13: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.

Epoch 13: val_loss improved from 0.02776 to 0.02776, saving model to final_convLSTM_shift3_best.h5


119/119 ━━━━━━━━━━━━━━━━━━━━ 4s 33ms/step - loss: 0.0252 - mae: 0.1798 - val_loss: 0.0278 - val_mae: 0.1907 - learning_rate: 0.0010
Epoch 14/100
119/119 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.0252 - mae: 0.1799
Epoch 14: val_loss did not improve from 0.02776
119/119 ━━━━━━━━━━━━━━━━━━━━ 4s 30ms/step - loss: 0.0252 - mae: 0.1799 - val_loss: 0.0278 - val_mae: 0.1907 - learning_rate: 5.0000e-04
Epoch 15/100
119/119 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 0.0252 - mae: 0.1798
Epoch 15: val_loss did not improve from 0.02776
119/119 ━━━━━━━━━━━━━━━━━━━━ 4s 35ms/step - loss: 0.0252 - mae: 0.1798 - val_loss: 0.0278 - val_mae: 0.1907 - learning_rate: 5.0000e-04
Epoch 16/100
119/119 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 0.0252 - mae: 0.1798
Epoch 16: val_loss did not improve from 0.02776
119/119 ━━━━━━━━━━━━━━━━━━━━ 4s 31ms/step - loss: 0.0252 - mae: 0.1798 - val_loss: 0.0278 - val_mae: 0.1907 - learning_rate: 5.0000e-04
Epoch 17/100
117/119 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 0

In [3]:
TEST_MONTH = "dataset/Sep_Validation_Data/INSAT_Validation_Data"


In [11]:
df = pd.read_csv("sept_forecast.csv")
print(df["predicted_GHI"].describe())


count    644.000000
mean     338.921173
std        0.000000
min      338.921173
25%      338.921173
50%      338.921173
75%      338.921173
max      338.921173
Name: predicted_GHI, dtype: float64


In [25]:
forecast_rows = []

for nc_path in tqdm(glob.glob(f"{TEST_MONTH}/**/*.nc", recursive=True), desc="Inferencing"):
    X_day, time_vec = load_day_nc_infer(nc_path, CHANNEL_VARS)
    if X_day is None: continue

    T = X_day.shape[0]
    past_ghi = np.zeros(T, dtype="float32")  # rolling GHI predictions

    for end_idx in range(WINDOW - 1, T):
        start_idx = end_idx - WINDOW + 1
        X_window = X_day[start_idx:end_idx + 1]

        # Construct rolling GHI input channel
        ghi_window = past_ghi[start_idx:end_idx + 1]
        ghi_stack = np.stack([np.full((5, 5), val, dtype="float32") for val in ghi_window])
        ghi_stack = ghi_stack[..., None]  # (WINDOW, 5, 5, 1)

        # Concatenate channels and normalize
        window = np.concatenate([X_window, ghi_stack], axis=-1)[None, ...]  # (1, WINDOW, 5, 5, C+1)
        window = window / SAT_SCALE

        # Predict
        pred_norm = model.predict(window, verbose=0)[0][0]
        pred_wm2 = float(pred_norm * MAX_GHI)
        pred_wm2 = max(pred_wm2, 0.0)

        # Update rolling GHI
        past_ghi[end_idx] = pred_wm2

        # Record prediction
        ts = pd.to_datetime(str(time_vec[end_idx]))
        forecast_rows.append({"timestamp": ts, "predicted_GHI": pred_wm2})


Inferencing: 100%|██████████| 14/14 [00:39<00:00,  2.81s/it]


In [26]:
import pandas as pd

# --- Convert forecast to DataFrame ---
df_pred = pd.DataFrame(forecast_rows)

# --- Sort & remove duplicates ---
df_pred = (
    df_pred
    .drop_duplicates("timestamp")
    .sort_values("timestamp")
    .set_index("timestamp")
)

# --- Create complete 15-min interval index for Sept 1–7 ---
full_index = pd.date_range("2024-09-01 00:00", "2024-09-07 23:45", freq="15min")
df_pred = df_pred.reindex(full_index)

# --- Interpolate any missing timestamps ---
missing = df_pred["predicted_GHI"].isna().sum()
if missing:
    print(f"{missing} missing slots – linearly interpolating.")
    df_pred["predicted_GHI"] = df_pred["predicted_GHI"].interpolate(method="time")

# --- Final clipping (for safety) ---
df_pred["predicted_GHI"] = df_pred["predicted_GHI"].clip(lower=0, upper=MAX_GHI)

# --- Save to CSV ---
df_pred.reset_index().rename(columns={"index": "timestamp"})\
      .to_csv("sept_forecast.csv", index=False)

print("✅  sept_forecast.csv written – rows:", len(df_pred))


315 missing slots – linearly interpolating.
✅  sept_forecast.csv written – rows: 672
